# TP n°4 - Feature selection
**Objectif:**

Ce TP vise à étudier quelques méthodes de sélection de variables (Filtrage) en utilisant la bibliothèque Scikit-learn.

[Documentation scikit-learn -> feature selection](https://scikit-learn.org/stable/modules/feature_selection.html)

Durant ce TP, on va aussi utiliser un modèle de réduction de variables (AFD).
Dans Scikit-learn, l'analyse factorielle discriminante (AFD) est mise en oeuvre dans la classe [LinearDiscriminantAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)

# Scikit-learn
Scikit-learn ou sklearn est une bibliothèque python libre conçue pour effectuer de l'apprentissage automatique.
Elle propose un set d'algorithmes de classification, régression et regroupement etc.

# Dataset
Pour illustrer le propos, on va utiliser le dataset `sales.csv` et nous allons ajouter des variables aléatoires qui représentent du bruit.


In [84]:
import pandas as pd
# Chargement du dataset sales.csv
df = pd.read_csv("../datasets/sales.csv")

In [85]:
# Afficher les dimension du dataset
df.shape
# Afficher les informations du dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   division            1000 non-null   object
 1   level of education  1000 non-null   object
 2   training level      1000 non-null   int64 
 3   work experience     1000 non-null   int64 
 4   salary              1000 non-null   int64 
 5   sales               1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


Pour l'exemple, on va supposer que la variable `level of education` (variable expliquée) contient les étiquettes des classes.

Pour référence, appliquons l'étape décisionnelle de l'analyse discriminante (comme modèle décisionnel) sur les données initiales et ensuite sur les données auxquelles les nouvelles variables aléatoires ont été ajoutées :

In [86]:
# Préparation des subsets d'entrainement et de test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Encoding des variables catégorielles
le = LabelEncoder()
df["division_encoded"] = le.fit_transform(df["division"])
df['level_education_encoded'] = le.fit_transform(df['level of education'])
# Ajout de 2 variables pour bruiter
np.random.seed(42)
df['noise1'] = np.random.rand(len(df))
df['noise2'] = np.random.rand(len(df))**5

In [87]:
df.head()

,division,level of education,training level,work experience,salary,sales,division_encoded,level_education_encoded,noise1,noise2
0,peripherals,some college,1,6,86465,374255,3,4,0.374540,0.000217
1,printers,associate's degree,1,8,94571,377561,4,0,0.950714,0.046730
2,printers,some college,1,11,110711,467281,4,4,0.731994,0.506917
3,office supplies,some college,2,0,60898,268977,2,4,0.598658,0.210486
4,printers,some college,2,9,103546,457900,4,4,0.156019,0.341339


In [88]:
X = df.drop(["level of education","division","level_education_encoded"], axis=1) # variables explicatives = élément de l'analyse
y = df["level_education_encoded"] # variable expliquée = objectif de l'analyse
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(f"dimension du subset d'entrainement: {X_train.shape}")
print(f"dimension du subset de test: {X_test.shape}")

dimension du subset d'entrainement: (700, 7)
dimension du subset de test: (300, 7)


In [89]:
X_train[0:2]

,training level,work experience,salary,sales,division_encoded,noise1,noise2
541,1,8,97748,377476,3,0.606175,0.000014
440,0,3,60608,135029,4,0.084838,0.858345


### Question
Comparer les performances du modèle entrainé sur les données initiales puis sur les données bruitées.

In [90]:
# Création du modèle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()

X_train_init = X_train.loc[::, ["training level","work experience","salary","sales","division_encoded"]]
X_test_init = X_test.loc[::, ["training level","work experience","salary","sales","division_encoded"]]
# entrainement avec les données initiales
model_init = model.fit(X_train_init, y_train)
accuracy_init = model_init.score(X_test_init, y_test)
# Prédiction sur le subset de test, pas obligatoire pour le TP
# predict_init = model_init.predict(X_test_init)
# entrainement avec les données bruitées
model_noise = model.fit(X_train, y_train)
accuracy_noise = model_noise.score(X_test, y_test)
# Prédiction sur le subset de test, pas obligatoire pour le TP
# predict_noise = model_noise.predict(X_test)
print(f"Accuracy: {accuracy_init:.2f}")
print(f"Accuracy: {accuracy_noise:.2f}")

Accuracy: 0.42
Accuracy: 0.47


### Correction


# Filtrage
La famille de filtrage est appliquée sans faire appel à un modèle prédictif.
Elle permet de maximiser l'information mutuelle entre la variable d'entrée et celle de sortie.
Elle minimise la redondance entre les variables d'entrée.
NB: Une coopération sous optimale avec le modèle prédictif qui n'intervient pas dans la sélection.
Cette approche présente un coût inférieur à celui de l'approche Wrapper.

## Suppression de variables à faible variance
La suppression de variables à faible variance consiste à éliminer les variables dont la variance est inférieure à un seuil (par défaut la valeur du seuil est 0, sont donc éliminées les variables constantes).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance)

Proposer une suppression de variables en utilisant le code ci-dessous

In [91]:
# Exemple de filtrage avec la suppression de variables à faible variance
from sklearn.feature_selection import VarianceThreshold
x_threshold = X_train.loc[::, ["training level","work experience","salary","sales","division_encoded","noise1","noise2"]]
selector = VarianceThreshold(threshold=2.0)

selector.fit_transform(x_threshold)
print(selector.get_feature_names_out())

['work experience' 'salary' 'sales']


In [92]:
# affichage des variances des différentes variables
for column, variance in zip(x_threshold.columns, selector.variances_):
    print(f"{column}: {variance:.2f}")

training level: 0.88
work experience: 8.34
salary: 324060303.96
sales: 12446796814.67
division_encoded: 1.73
noise1: 0.09
noise2: 0.07


In [73]:
# Fixer un seuil de variance à 10 pour éliminer quelques variables
selector = VarianceThreshold(threshold=10)
selector.fit_transform(x_threshold)
print(selector.get_feature_names_out())

['salary' 'sales']


In [ ]:
# Affichage des variables sélectionnées
# TODO

### Question
Commenter la variance des variables ajoutées (pour `threshold=10`)
A travers une autre combinaison de variables, proposer une 2ème classification via le même modèle étudié `from sklearn.discriminant_analysis import LinearDiscriminantAnalysis`.
Commenter le résultat obtenu.

### Réponse
Les variables salary et sales apportent beaucoup d'informations étant donné qu'elles ont une grande variance. Les variables training level, work experience et division_encoded n'apportent pas beaucoup d'informations en vu de leur faible variance.

### Correction


In [ ]:
model = LinearDiscriminantAnalysis()
# TODO

## Sélection univariée
La sélection univariées cherche à déterminer (à travers des tests statistiques comme le test de `Chi2`) dans quelle mesure chaque variable d'entrée "explique" la variable de sortie; les variables les moins explicatives individuellement sont éliminées.

Cette méthode élimine les variables pour lesquelles les valeurs de l'information mutuelle avec la variable de sortie sont les plus faibles (c'est à dire, qui « expliquent » le moins bien la variable de sortie). Nous avons l'intention de garder la moitié des variables et utilisons donc la fonction `SelectKBest` (d'autres sont disponibles, voir la documentation).

[Doc. sélection univariée](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)
[Doc. information mutuelle pour la classification](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

### Question
Utilisez la sélection univariée pour sélectionner les meilleurs groupe de variables qui explique le mieux la variable expliquée.

### Correction

In [94]:
# Exemple de filtrage avec la sélection univariée
from sklearn.feature_selection import SelectKBest, f_oneway
selector = SelectKBest(score_func=f_oneway, k=3)
selector.fit_transform(X_train, y_train)
print(selector.get_support())
print(selector.get_feature_names_out())

[False  True  True  True False False False]
['work experience' 'salary' 'sales']


### Question
Que constatez-vous par rapport aux variables ajoutées (bruit) ?

### Correction

### Question
Décrire la performance du modèle décisionnel. Comparer aux résultats obtenus par la méthode précédente.

### Correction

# Wrapper
La famille Wrapper coopère directement avec le modèle prédictif.
Choix des variables qui maximisent les performances du modèle.
**Inconvénients :**
- Coût élevé
- Pas de justification théorique de la sélection des variables
- Incompréhension des relations de dépendances entre les variables.
- La procédure de sélection est spécifique au modèle utilisé.

## Sélection séquentielle
Un modèle décisionnel doit être développé sur chaque (sous-)ensemble candidat de variables d'entrée et c'est la performance du modèle qui caractérise le (sous-)ensemble de variable. Deux versions sont proposées, une incrémentale (*Forward-SFS*) et une décrémentale (*Backward-SFS*).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#sequential-feature-selection)

## Elimination récursive de variables
**Principe:** apprentissage du modèle sur la totalité des variables d'entrée pour extraire la pertinence de chaque variable (variance).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)

## Sélection avec SelectFromModel
**Principe:** apprentissage du modèle sur la totalité des variables d'entrée pour extraire la pertinence de chaque variable (variance).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel)

# Embedding
L'embedding (ou l'intégration) est une opération de sélection qui est intégrée à la méthode de construction de modèle. Pas de sur-coût par rapport à la construction du modèle mais cette approche ne peut pas être utilisée avec tout type de modèle.


# Exercice
Quelles sont les meilleures caractéristiques du dataset `sales.csv` qui permettent de prédire au mieux le salaire ?
**Note:** Vous pouvez utiliser ce modèle: `from sklearn.linear_model import LinearRegression`


# TP suivant ?
TP n°6: La réduction des variables par réduction des dimensions --> ACP.
